In [7]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
df = pd.read_csv("datasetRAW.csv", sep = ",")

In [8]:
df = df[df['Target'] != 'Enrolled']

### Primeiro Teste removendo a coluna Nacionalidade

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = df[df['Target'] != 'Enrolled']
x = df.drop(columns=['Target', 'Nacionality'])
y = df['Target'].apply(lambda x: 1 if x == 'Dropout' else 0)

x.shape, y.shape

#Modelo
modelo = LinearRegression()

#Normalização dos dados para que o algoritmo não de mais importância a valores como idade = 70 em vez de Creaditos aprovados = 18
scaler = StandardScaler()
x_scaled = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)

# Inicializar StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#Listas para registar os resultados para depois serem feitas as médias
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for i_train, i_test in kf.split(x_scaled, y):
    x_train, x_test = x_scaled.iloc[i_train], x_scaled.iloc[i_test] #iloc permite usar linhas/colunas especificas do data set conforme o index
    y_train, y_test = y.iloc[i_train], y.iloc[i_test]

    #Treino do modelo
    modelo.fit(x_train, y_train)

    #Resultado de valores continuos uma vez que estamos a usar regressão linear
    res_continuos = modelo.predict(x_test) #Retorna apenas os resultados da previsão

    #Converter de valores continuos para Valores Binários, problema de regressão para problema de classificação
    intervalo_de_admissao = 0.5
    res_binario = (res_continuos >= intervalo_de_admissao).astype(int)

    #Calculo de Metricas
    accuracy_list.append(accuracy_score(y_test, res_binario))
    precision_list.append(precision_score(y_test, res_binario))
    recall_list.append(recall_score(y_test, res_binario))
    f1_list.append(f1_score(y_test, res_binario))

print(f"Acurácia média: {np.mean(accuracy_list):.4f}")
print(f"Precisão média: {np.mean(precision_list):.4f}")
print(f"Recall médio: {np.mean(recall_list):.4f}")
print(f"F1-score médio: {np.mean(f1_list):.4f}")


Acurácia média: 0.9072
Precisão média: 0.9581
Recall médio: 0.7980
F1-score médio: 0.8706
